# Resampling Methods

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import statsmodels.api as sm

**1. Validation Default Dataset**

In [6]:
default_df = pd.read_csv(r'C:\Users\caine\OneDrive\Documents\statistical_learning_problems\0 datasets\Default.csv', index_col=0)

In [7]:
default_df.head()

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


*(a) Fit a logistic regression model that uses `income` and `balance` to predict `default`.*

In [ ]:
def_lr = sm.GLM()

*(b) Using the validation set approach, estimate the test error of this model.*

*(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.*

*(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.*